# Project - Stock Information AI Assistant


In [1]:
!pip install finnhub-python

In [2]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import finnhub
from typing import Dict, List, Any, Optional
from datetime import datetime

In [3]:
import logging

# Configure root logger
logging.basicConfig(
    level=logging.INFO,              # Set level: DEBUG, INFO, WARNING, ERROR
    format="%(asctime)s [%(levelname)s] %(message)s", 
    force=True                       # Ensures reconfiguration if you rerun this cell
)

logger = logging.getLogger(__name__)  # Use a global logger object
logger.info("Logger initialized!")

2025-08-29 02:28:59,653 [INFO] Logger initialized!


In [4]:
# Initialization

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
FINNHUB_API_KEY = os.getenv("FINNHUB_API_KEY")

if openai_api_key:
    logger.info(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    logger.error("OpenAI API Key not set")

if FINNHUB_API_KEY:
    logger.info(f"FINNHUB_API_KEY exists!")
else:
    logger.error("OpenAI API Key not set")
    
MODEL = "gpt-5-mini"
openai = OpenAI()
finnhub_client = finnhub.Client(api_key=FINNHUB_API_KEY)

2025-08-29 02:28:59,657 [INFO] OpenAI API Key exists and begins sk-proj-
2025-08-29 02:28:59,657 [INFO] FINNHUB_API_KEY exists!


In [5]:
system_message =  f"""
You are called "TickerBot", You are a helpful stock information assistant specializing in US stocks. You provide factual, educational information without investment advice. You have access to tools for:
- Stock symbol lookup
- Real-time quotes
- Company Financials
- Company News
- Market overview

### **Core Principles**
**Educational Focus**: Explain financial metrics clearly and keep an educational tone.
**Factual**: NEVER give buy/sell advice or predictions.
**Accurate always**: If no data is available, inform the user in a friendly way. Always be accurate. If you don't know the answer, simply say so. Do not make up your own stock details information.

### **How to Handle Different Requests**
- For all temporal reasoning in this chat you can use `get_current_time()` tool to get time and then relative to current time you can proceed.
- When users mention companies, search for symbols with the tool `search_symbol()` else proceed directly if obvious match
- Try to search for news or data for only for a maximum of 1 month time range, else it becomes a lot of data to parse. If user asks for recent news just check the last 5 days from today; For example if today is 10-06-2025, use from="2025-06-05", to="2025-06-10"

**Market Overview Requests**:
- "What's happening in the market?" → Use `get_market_overview("general")`
- Summarize all news stories with very brief analysis

### **Error Handling**
- If symbol search fails: "I couldn't find that company in US markets. Could you try a different name or provide the ticker symbol?"
- If some information gathered from the tool call says unavailable or error do not present it to the user unless they had specifically asked for it. Present rest of the gathered information if any.
- If data is unavailable: "Some data isn't available right now, but here's what I found..."
- Stay helpful and suggest alternatives
"""

In [6]:
def get_current_time() -> Dict[str, Any]:
    """
    Retrieve the current UTC time in ISO format with timezone.
    Returns a dictionary for consistency with other tools.
    """
    try:
        current_time = datetime.utcnow().isoformat() + 'Z'
        return {
            "success": True,
            "current_time": current_time
        }
    except Exception as e:
        return {"success": False, "error": f"Failed to get time: {str(e)[:100]}"}

In [7]:
get_current_time_function = {
    "name": "get_current_time",
    "description": "Get the current UTC time in ISO format (YYYY-MM-DDTHH:MM:SS.ssssssZ). Useful for temporal reasoning, date calculations, or setting time ranges for queries like news.",
    "parameters": {
        "type": "object",
        "properties": {},  # No parameters needed
        "required": []
    }
}
get_current_time_tool = {"type": "function", "function": get_current_time_function}

In [8]:
def validate_symbol(symbol: str) -> bool:
    """Validate stock symbol format"""
    if not symbol or not isinstance(symbol, str):
        return False
    return symbol.isalnum() and 1 <= len(symbol) <= 5 and symbol.isupper()

def search_symbol(query: str) -> Dict[str, Any]:
    """Search for stock symbol using Finnhub client"""
    logger.info(f"Tool search_symbol called for {query}")
    try:
        if not query or len(query.strip()) < 1:
            return {"success": False, "error": "Invalid search query"}
        
        query = query.strip()[:50]
        result = finnhub_client.symbol_lookup(query)
        logger.info(f"Tool search_symbol {result}")
        
        if result.get("result") and len(result["result"]) > 0:
            first_result = result["result"][0]
            symbol = first_result.get("symbol", "").upper()
            
            if validate_symbol(symbol):
                return {
                    "success": True,
                    "symbol": symbol
                }
            else:
                return {"success": False, "error": "Invalid symbol format found"}
        else:
            return {"success": False, "error": "No matching US stocks found"}
            
    except Exception as e:
        return {"success": False, "error": f"Symbol search failed: {str(e)[:100]}"}

In [9]:
search_symbol_function = {
    "name": "search_symbol",
    "description": "Search for a stock symbol / ticker symbol based on company name or partial name",
    "parameters": {
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "Company name or partial name to search for, extract only relevant name part and pass it here, keep this to less than 50 characters"
            }
        },
        "required": [
            "query"
        ]
    }
}

search_symbol_tool = {"type": "function", "function": search_symbol_function}

In [10]:
def _format_big_number_from_millions(value_millions: Any) -> str:
    """
    Finnhub returns some large metrics (marketCapitalization, enterpriseValue, revenueTTM)
    in MILLIONS USD. Convert to full USD and format with M/B/T suffixes.
    """
    if value_millions is None:
        return "Unavailable"
    try:
        value = float(value_millions) * 1_000_000  # convert millions -> full USD
    except (TypeError, ValueError):
        return "Unavailable"

    trillion = 1_000_000_000_000
    billion = 1_000_000_000
    million = 1_000_000

    if value >= trillion:
        return f"{value / trillion:.2f}T USD"
    if value >= billion:
        return f"{value / billion:.2f}B USD"
    if value >= million:
        return f"{value / million:.2f}M USD"
    return f"{value:.2f} USD"


def _safe_metric(metrics: Dict[str, Any], key: str) -> Any:
    """
    Return metric value if present; otherwise "Unavailable".
    We intentionally return the raw value for numeric metrics (no rounding/format)
    except for the specially formatted big-number fields handled elsewhere.
    """
    if metrics is None:
        return "Unavailable"
    val = metrics.get(key)
    return val if val is not None else "Unavailable"


def get_company_financials(symbol: str) -> Dict[str, Any]:
    """
    Fetch and return a curated set of 'basic' financial metrics for `symbol`.
    - Calls finnhub_client.company_basic_financials(symbol, 'all')
    - Formats market cap, enterprise value, revenue (Finnhub returns these in millions)
    - Returns success flag and readable keys
    """
    logger.info(f"Tool get_company_financials called for {symbol}")
    try:
        if not symbol or not symbol.strip():
            return {"success": False, "error": "Invalid stock symbol"}

        symbol = symbol.strip().upper()

        # --- API Call ---
        financials_resp = finnhub_client.company_basic_financials(symbol, "all")
        logger.info(f"Tool company_basic_financials {financials_resp}")

        # Finnhub places primary values under "metric"
        metrics = financials_resp.get("metric", {})
        if not metrics:
            return {"success": False, "error": "No financial metrics found"}

        # --- Build result using helpers ---
        result = {
            "success": True,
            "symbol": symbol,
            "financials": {
                "Market Cap": _format_big_number_from_millions(metrics.get("marketCapitalization")),
                "Enterprise Value": _format_big_number_from_millions(metrics.get("enterpriseValue")),
                "P/E Ratio (TTM)": _safe_metric(metrics, "peBasicExclExtraTTM"),
                "Forward P/E": _safe_metric(metrics, "forwardPE"),
                "Revenue (TTM)": _format_big_number_from_millions(metrics.get("revenueTTM")),
                "Gross Margin (TTM)": _safe_metric(metrics, "grossMarginTTM"),
                "Net Profit Margin (TTM)": _safe_metric(metrics, "netProfitMarginTTM"),
                "EPS (TTM)": _safe_metric(metrics, "epsTTM"),
                "EPS Growth (5Y)": _safe_metric(metrics, "epsGrowth5Y"),
                "Dividend Yield (Indicated Annual)": _safe_metric(metrics, "dividendYieldIndicatedAnnual"),
                "Current Ratio (Quarterly)": _safe_metric(metrics, "currentRatioQuarterly"),
                "Debt/Equity (Long Term, Quarterly)": _safe_metric(metrics, "longTermDebt/equityQuarterly"),
                "Beta": _safe_metric(metrics, "beta"),
                "52-Week High": _safe_metric(metrics, "52WeekHigh"),
                "52-Week Low": _safe_metric(metrics, "52WeekLow"),
            }
        }

        return result

    except Exception as e:
        # keep error message short but useful for debugging
        return {"success": False, "error": f"Failed to fetch metrics: {str(e)[:200]}"}

In [11]:
get_company_financials_function = {
    "name": "get_company_financials",
    "description": "Fetch and return a curated set of basic financial metrics for a stock symbol. Calls Finnhub's company_basic_financials API, formats large numbers (market cap, enterprise value, revenue) in M/B/T USD, and shows metrics like P/E ratios, EPS, margins, dividend yield, debt/equity, beta, and 52-week range. Returns 'Unavailable' for missing values.",
    "parameters": {
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "Stock ticker symbol to fetch metrics for. Example: 'AAPL' for Apple Inc."
            }
        },
        "required": [
            "symbol"
        ]
    }
}


get_company_financials_tool = {"type": "function", "function": get_company_financials_function}

In [12]:
def get_stock_quote(symbol: str) -> dict:
    """
    Fetch the latest stock quote for a given ticker symbol using Finnhub's /quote endpoint.
    Returns current price, daily high/low, open, previous close, percent change, and readable timestamp.
    """
    logger.info(f"Tool get_stock_quote called for {symbol}")
    try:
        if not symbol or len(symbol.strip()) < 1:
            return {"success": False, "error": "Invalid symbol provided"}
        
        symbol = symbol.strip().upper()
        data = finnhub_client.quote(symbol)
        logger.info(f"Tool get_stock_quote {data}")

        if not data or "c" not in data:
            return {"success": False, "error": "No quote data found"}
        
        # Convert epoch timestamp to ISO UTC if present
        timestamp = data.get("t")
        if timestamp and isinstance(timestamp, (int, float)):
            timestamp = datetime.utcfromtimestamp(timestamp).isoformat() + "Z"
        else:
            timestamp = "Unavailable"
        
        return {
            "success": True,
            "symbol": symbol,
            "current_price": round(data.get("c", 0), 2) if data.get("c") is not None else "Unavailable",
            "change": round(data.get("d", 0), 2) if data.get("d") is not None else "Unavailable",
            "percent_change": f"{round(data.get('dp', 0), 2)}%" if data.get("dp") is not None else "Unavailable",
            "high_price": round(data.get("h", 0), 2) if data.get("h") is not None else "Unavailable",
            "low_price": round(data.get("l", 0), 2) if data.get("l") is not None else "Unavailable",
            "open_price": round(data.get("o", 0), 2) if data.get("o") is not None else "Unavailable",
            "previous_close": round(data.get("pc", 0), 2) if data.get("pc") is not None else "Unavailable",
            "timestamp": timestamp
        }
    except Exception as e:
        return {"success": False, "error": f"Quote retrieval failed: {str(e)[:100]}"}

In [13]:
get_stock_quote_function = {
  "name": "get_stock_quote",
  "description": "Retrieve the latest stock quote for a given symbol, including current price, daily high/low, open, previous close, and percent change. Data is near real-time. Avoid constant polling; use websockets for streaming updates.",
  "parameters": {
    "type": "object",
    "properties": {
      "symbol": {
        "type": "string",
        "description": "Stock ticker symbol to fetch the latest quote for. Example: 'AAPL', 'MSFT'."
      }
    },
    "required": ["symbol"]
  }
}

get_stock_quote_tool = {"type": "function", "function": get_stock_quote_function}


In [14]:
def get_company_news(symbol: str, _from: str, to: str):
    """
    Fetch the top 5 latest company news for a stock symbol within a date range.
    - Ensures the range does not exceed ~1 months (35 days).
    - Best practice: Keep searches to a month or less to avoid too much data.

    Args:
        symbol (str): Stock ticker (e.g., "AAPL").
        _from (str): Start date in YYYY-MM-DD format.
        to (str): End date in YYYY-MM-DD format.

    Returns:
        list or dict: Cleaned news data or error message.
    """
    # Validate date format
    logger.info(f"Tool get_company_news called for {symbol} from {_from} to {to}")
    try:
        start_date = datetime.strptime(_from, "%Y-%m-%d")
        end_date = datetime.strptime(to, "%Y-%m-%d")
    except ValueError:
        return {"success": False, "error": "Invalid date format. Use YYYY-MM-DD."}

    # Check date range
    delta_days = (end_date - start_date).days
    if delta_days > 35:
        return {
            "success": False, 
            "error": f"Date range too large ({delta_days} days). "
                     "Please use a range of 1 months or less."
        }

    # Fetch data
    try:
        news = finnhub_client.company_news(symbol, _from=_from, to=to)
        logger.info(f"Tool get_company_news {news}")
    except Exception as e:
        return {"success": False, "error": str(e)}

    # Do not want to report just the latest news in the time period
    if len(news) <= 10:
    # If 10 or fewer articles, take all
        selected_news = news
    else:
        # Take first 5 (oldest) and last 5 (newest)
        selected_news = news[:5] + news[-5:]

    # Clean & transform objects
    cleaned_news = []
    for article in selected_news:
        cleaned_news.append({
            "summary": article.get("summary"),
            "source": article.get("source"),
            "published_at": datetime.utcfromtimestamp(article["datetime"]).strftime("%Y-%m-%d %H:%M:%S UTC"),
            "related": article.get("related")
        })

    return {"success": True, "news": cleaned_news}

In [15]:
get_company_news_function = {
    "name": "get_company_news",
    "description": "Fetch the top 5 most recent company news articles for a given stock symbol. ⚠️ Avoid querying more than a 1-month range at a time as it may return too much data. Only tells news about company within last 1 year. An error is returned if the requested time range exceeds 1 month.",
    "parameters": {
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "Stock ticker symbol, e.g., 'AAPL'."
            },
            "_from": {
                "type": "string",
                "description": "Start date in YYYY-MM-DD format. Ensure it is not more than 1 year ago from today. Ensure it is before or equal to the date in to."
            },
            "to": {
                "type": "string",
                "description": "End date in YYYY-MM-DD format. Ensure it is not more than 1 year ago. Ensure it is after or equal to the date in from."
            }
        },
        "required": [
            "symbol",
            "_from",
            "to"
        ]
    }
}

get_company_news_tool = {"type": "function", "function": get_company_news_function}

In [16]:
def get_market_news(category: str = "general"):
    """
    Fetch the latest market news for a given category.
    - Returns only the top 7 articles.

    Args:
        category (str): News category. One of ["general", "forex", "crypto", "merger"].

    Returns:
        list or dict: A cleaned list of news articles or error message.
    """
    logger.info(f"Tool get_market_news called for category '{category}'")

    try:
        news = finnhub_client.general_news(category)
        logger.info(f"Tool get_market_news {news}")
    except Exception as e:
        return {"success": False, "error": str(e)}

    # Do not want to report just the latest news in the time period
    if len(news) <= 10:
    # If 10 or fewer articles, take all
        selected_news = news
    else:
        # Take first 5 (oldest) and last 5 (newest)
        selected_news = news[:5] + news[-5:]

    # Clean & transform objects
    cleaned_news = []
    for article in selected_news:
        cleaned_news.append({
            "headline": article.get("headline"),
            "summary": article.get("summary"),
            "source": article.get("source"),
            "category": article.get("category"),
            "published_at": datetime.utcfromtimestamp(article["datetime"]).strftime("%Y-%m-%d %H:%M:%S UTC"),
            "related": article.get("related")
        })

    return {"success": True, "news": cleaned_news}

In [17]:
get_market_news_function = {
  "name": "get_market_news",
  "description": "Fetch the latest market news by category. Returns the top 10 news articles with headline, summary, source, category, published time (UTC), and URLs. Categories: general, forex, crypto, merger. Use this to quickly get relevant financial news.",
  "parameters": {
    "type": "object",
    "properties": {
      "category": {
        "type": "string",
        "description": "News category to fetch. One of: general, forex, crypto, merger."
      }
    },
    "required": ["category"]
  }
}

get_market_news_tool = {"type": "function", "function": get_market_news_function}

In [18]:
# List of tools:
tools = [search_symbol_tool, get_company_financials_tool, get_stock_quote_tool, get_company_news_tool, get_market_news_tool, get_current_time_tool]
tool_functions = {
    "search_symbol": search_symbol,
    "get_company_financials": get_company_financials,
    "get_stock_quote": get_stock_quote,
    "get_company_news": get_company_news,
    "get_market_news": get_market_news,
    "get_current_time": get_current_time
}

## Getting OpenAI to use our Tool

There's some fiddly stuff to allow OpenAI "to call our tool"

What we actually do is give the LLM the opportunity to inform us that it wants us to run the tool.

Here's how the new chat function looks:

In [19]:
def execute_tool_call(tool_call):
    func_name = tool_call.function.name
    args = json.loads(tool_call.function.arguments)

    logger.info(f"Executing tool: {func_name} with args: {args}")

    func = tool_functions.get(func_name)
    if not func:
        result = {"error": f"Function '{func_name}' not found"}
    else:
        try:
            result = func(**args)
        except Exception as e:
            logger.exception(f"Error executing {func_name}")
            result = {"error": str(e)}

    return {
        "role": "tool",
        "tool_call_id": tool_call.id,
        "content": json.dumps(result)
    }

In [20]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    while True:
        # Send to OpenAI
        response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
        ai_message = response.choices[0].message
        finish_reason = response.choices[0].finish_reason

        # If no tool calls, this is user-facing output
        if finish_reason != "tool_calls":
            return ai_message.content  # ✅ Only return final assistant content

        # Otherwise, handle all tool calls in this message
        tool_responses = []
        for tool_call in ai_message.tool_calls:
            tool_responses.append(execute_tool_call(tool_call))

        # Add tool call request + tool responses to conversation
        messages.append(ai_message)
        messages.extend(tool_responses)

In [ ]:
gr.ChatInterface(fn=chat, type="messages", title="TickerBot", description="Ask about stock prices, company profiles and market news!").launch(debug=True)

2025-08-29 02:29:05,739 [INFO] HTTP Request: GET http://127.0.0.1:7860/gradio_api/startup-events "HTTP/1.1 200 OK"
2025-08-29 02:29:05,746 [INFO] HTTP Request: HEAD http://127.0.0.1:7860/ "HTTP/1.1 200 OK"


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


2025-08-29 02:29:07,097 [INFO] HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"
2025-08-29 02:29:17,018 [INFO] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-29 02:29:17,024 [INFO] Executing tool: search_symbol with args: {'query': 'Apple'}
2025-08-29 02:29:17,025 [INFO] Tool search_symbol called for Apple
2025-08-29 02:29:17,777 [INFO] Tool search_symbol {'count': 11, 'result': [{'description': 'Apple Inc', 'displaySymbol': 'AAPL', 'symbol': 'AAPL', 'type': 'Common Stock'}, {'description': 'Apple Hospitality REIT Inc', 'displaySymbol': 'APLE', 'symbol': 'APLE', 'type': 'Common Stock'}, {'description': 'Apple iSports Group Inc', 'displaySymbol': 'AAPI', 'symbol': 'AAPI', 'type': 'Common Stock'}, {'description': 'Apple Flavor & Fragrance Group Co Ltd', 'displaySymbol': '603020.SS', 'symbol': '603020.SS', 'type': 'Common Stock'}, {'description': 'Maui Land & Pineapple Company Inc', 'displaySymbol': 'MLP', 'symbol': 'MLP', 'ty

# Let's go multi-modal!!

We can use DALL-E-3, the image generation model behind GPT-4o, to make us some images

Let's put this in a function called artist.

### Price alert: each time I generate an image it costs about 4 cents - don't go crazy with images!

In [ ]:
# Some imports for handling images

import base64
from io import BytesIO
from PIL import Image

In [ ]:
def artist(city):
    image_response = openai.images.generate(
            model="dall-e-3",
            prompt=f"An image representing a vacation in {city}, showing tourist spots and everything unique about {city}, in a vibrant pop-art style",
            size="1024x1024",
            n=1,
            response_format="b64_json",
        )
    image_base64 = image_response.data[0].b64_json
    image_data = base64.b64decode(image_base64)
    return Image.open(BytesIO(image_data))

In [ ]:
image = artist("New York City")
display(image)

## Audio (NOTE - Audio is optional for this course - feel free to skip Audio if it causes trouble!)

And let's make a function talker that uses OpenAI's speech model to generate Audio

### Troubleshooting Audio issues

If you have any problems running this code below (like a FileNotFound error, or a warning of a missing package), you may need to install FFmpeg, a very popular audio utility.

**For PC Users**

Detailed instructions are [here](https://chatgpt.com/share/6724efee-6b0c-8012-ac5e-72e2e3885905) and summary instructions:

1. Download FFmpeg from the official website: https://ffmpeg.org/download.html

2. Extract the downloaded files to a location on your computer (e.g., `C:\ffmpeg`)

3. Add the FFmpeg bin folder to your system PATH:
- Right-click on 'This PC' or 'My Computer' and select 'Properties'
- Click on 'Advanced system settings'
- Click on 'Environment Variables'
- Under 'System variables', find and edit 'Path'
- Add a new entry with the path to your FFmpeg bin folder (e.g., `C:\ffmpeg\bin`)
- Restart your command prompt, and within Jupyter Lab do Kernel -> Restart kernel, to pick up the changes

4. Open a new command prompt and run this to make sure it's installed OK
`ffmpeg -version`

**For Mac Users**

1. Install homebrew if you don't have it already by running this in a Terminal window and following any instructions:  
`/bin/bash -c "$(curl -fsSL https://raw.githubusercontent.com/Homebrew/install/HEAD/install.sh)"`

2. Then install FFmpeg with `brew install ffmpeg`

3. Verify your installation with `ffmpeg -version` and if everything is good, within Jupyter Lab do Kernel -> Restart kernel to pick up the changes

Message me or email me at ed@edwarddonner.com with any problems!

## To check you now have ffmpeg and can access it here

Excecute the next cell to see if you get a version number. (Putting an exclamation mark before something in Jupyter Lab tells it to run it as a terminal command rather than python code).

If this doesn't work, you may need to actually save and close down your Jupyter lab, and start it again from a new Terminal window (Mac) or Anaconda prompt (PC), remembering to activate the llms environment. This ensures you pick up ffmpeg.

And if that doesn't work, please contact me!

In [ ]:
!ffmpeg -version
!ffprobe -version
!ffplay -version

# For Mac users - and possibly many PC users too

This version should work fine for you. It might work for Windows users too, but you might get a Permissions error writing to a temp file. If so, see the next section!

As always, if you have problems, please contact me! (You could also comment out the audio talker() in the later code if you're less interested in audio generation)

In [ ]:
from pydub import AudioSegment
from pydub.playback import play

def talker(message):
    response = openai.audio.speech.create(
      model="tts-1",
      voice="onyx",    # Also, try replacing onyx with alloy
      input=message
    )
    
    audio_stream = BytesIO(response.content)
    audio = AudioSegment.from_file(audio_stream, format="mp3")
    play(audio)

In [ ]:
talker("Well, hi there")

# For Windows users (or any Mac users with problems above)

## First try the Mac version above, but if you get a permissions error writing to a temp file, then this code should work instead.

A collaboration between students Mark M. and Patrick H. and Claude got this resolved!

Below are 4 variations - hopefully one of them will work on your PC. If not, message me please!

And for Mac people - all 3 of the below work on my Mac too - please try these if the Mac version gave you problems.

## PC Variation 1

In [ ]:
import base64
from io import BytesIO
from PIL import Image
from IPython.display import Audio, display

def talker(message):
    response = openai.audio.speech.create(
        model="tts-1",
        voice="onyx",
        input=message)

    audio_stream = BytesIO(response.content)
    output_filename = "output_audio.mp3"
    with open(output_filename, "wb") as f:
        f.write(audio_stream.read())

    # Play the generated audio
    display(Audio(output_filename, autoplay=True))

talker("Well, hi there")

## PC Variation 2

In [ ]:
import tempfile
import subprocess
from io import BytesIO
from pydub import AudioSegment
import time

def play_audio(audio_segment):
    temp_dir = tempfile.gettempdir()
    temp_path = os.path.join(temp_dir, "temp_audio.wav")
    try:
        audio_segment.export(temp_path, format="wav")
        time.sleep(3) # Student Dominic found that this was needed. You could also try commenting out to see if not needed on your PC
        subprocess.call([
            "ffplay",
            "-nodisp",
            "-autoexit",
            "-hide_banner",
            temp_path
        ], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    finally:
        try:
            os.remove(temp_path)
        except Exception:
            pass
 
def talker(message):
    response = openai.audio.speech.create(
        model="tts-1",
        voice="onyx",  # Also, try replacing onyx with alloy
        input=message
    )
    audio_stream = BytesIO(response.content)
    audio = AudioSegment.from_file(audio_stream, format="mp3")
    play_audio(audio)

talker("Well hi there")

## PC Variation 3

In [ ]:
import os
from pydub import AudioSegment
from pydub.playback import play
from io import BytesIO

def talker(message):
    # Set a custom directory for temporary files on Windows
    custom_temp_dir = os.path.expanduser("~/Documents/temp_audio")
    os.environ['TEMP'] = custom_temp_dir  # You can also use 'TMP' if necessary
    
    # Create the folder if it doesn't exist
    if not os.path.exists(custom_temp_dir):
        os.makedirs(custom_temp_dir)
    
    response = openai.audio.speech.create(
        model="tts-1",
        voice="onyx",  # Also, try replacing onyx with alloy
        input=message
    )
    
    audio_stream = BytesIO(response.content)
    audio = AudioSegment.from_file(audio_stream, format="mp3")

    play(audio)

talker("Well hi there")

## PC Variation 4

### Let's try a completely different sound library

First run the next cell to install a new library, then try the cell below it.

In [ ]:
!pip install simpleaudio

In [ ]:
from pydub import AudioSegment
from io import BytesIO
import tempfile
import os
import simpleaudio as sa

def talker(message):
    response = openai.audio.speech.create(
        model="tts-1",
        voice="onyx",  # Also, try replacing onyx with alloy
        input=message
    )
    
    audio_stream = BytesIO(response.content)
    audio = AudioSegment.from_file(audio_stream, format="mp3")

    # Create a temporary file in a folder where you have write permissions
    with tempfile.NamedTemporaryFile(suffix=".wav", delete=False, dir=os.path.expanduser("~/Documents")) as temp_audio_file:
        temp_file_name = temp_audio_file.name
        audio.export(temp_file_name, format="wav")
    
    # Load and play audio using simpleaudio
    wave_obj = sa.WaveObject.from_wave_file(temp_file_name)
    play_obj = wave_obj.play()
    play_obj.wait_done()  # Wait for playback to finish

    # Clean up the temporary file afterward
    os.remove(temp_file_name)
    
talker("Well hi there")

## PC Users - if none of those 4 variations worked!

Please get in touch with me. I'm sorry this is causing problems! We'll figure it out.

Alternatively: playing audio from your PC isn't super-critical for this course, and you can feel free to focus on image generation and skip audio for now, or come back to it later.

# Our Agent Framework

The term 'Agentic AI' and Agentization is an umbrella term that refers to a number of techniques, such as:

1. Breaking a complex problem into smaller steps, with multiple LLMs carrying out specialized tasks
2. The ability for LLMs to use Tools to give them additional capabilities
3. The 'Agent Environment' which allows Agents to collaborate
4. An LLM can act as the Planner, dividing bigger tasks into smaller ones for the specialists
5. The concept of an Agent having autonomy / agency, beyond just responding to a prompt - such as Memory

We're showing 1 and 2 here, and to a lesser extent 3 and 5. In week 8 we will do the lot!

In [ ]:
def chat(history):
    messages = [{"role": "system", "content": system_message}] + history
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    image = None
    
    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        image = artist(city)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
        
    reply = response.choices[0].message.content
    history += [{"role":"assistant", "content":reply}]

    # Comment out or delete the next line if you'd rather skip Audio for now..
    talker(reply)
    
    return history, image

In [ ]:
# More involved Gradio code as we're not using the preset Chat interface!
# Passing in inbrowser=True in the last line will cause a Gradio window to pop up immediately.

with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=500, type="messages")
        image_output = gr.Image(height=500)
    with gr.Row():
        entry = gr.Textbox(label="Chat with our AI Assistant:")
    with gr.Row():
        clear = gr.Button("Clear")

    def do_entry(message, history):
        history += [{"role":"user", "content":message}]
        return "", history

    entry.submit(do_entry, inputs=[entry, chatbot], outputs=[entry, chatbot]).then(
        chat, inputs=chatbot, outputs=[chatbot, image_output]
    )
    clear.click(lambda: None, inputs=None, outputs=chatbot, queue=False)

ui.launch(inbrowser=True)

# Exercises and Business Applications

Add in more tools - perhaps to simulate actually booking a flight. A student has done this and provided their example in the community contributions folder.

Next: take this and apply it to your business. Make a multi-modal AI assistant with tools that could carry out an activity for your work. A customer support assistant? New employee onboarding assistant? So many possibilities! Also, see the week2 end of week Exercise in the separate Notebook.

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a HUGE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>